# Проект для интернет магазина

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

Интернет-магазин запускает новый сервис. Пользователи смогут редактировать и дополнять описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 



Метрика: модель со значением метрики качества *F1* не меньше 0.75. 



## Подготовка

In [1]:
import nltk
import transformers
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.metrics import f1_score, mean_absolute_error, make_scorer
import spacy
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize 

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
#  Загрузили и посмотрели на баланс классов 
data = pd.read_csv('/datasets/toxic_comments.csv')
data = data.drop_duplicates().reset_index(drop=True)
print(data.info())
print(data['toxic'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None
0    143346
1     16225
Name: toxic, dtype: int64


In [3]:
lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''





In [4]:
data_new =  data.sample(n=80000, random_state=12345).reset_index(drop=True)

corpus = data_new['text']
toxic = data_new['toxic']

In [5]:
# Функция лемматизации
def lemmatize(corpus):
    corpus_new = []
    for i in range(len(corpus)): 
        text = str(corpus[i])
#         print(type(text))
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(text))  
        wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag == '': 
                res_words.append(word)
            else:
                res_words.append(lemmatizer.lemmatize(word, tag))   
        res_words =  " ".join(res_words)  
            
        corpus_new.append(res_words)
            
    return corpus_new


Соединил 2 датафрейма с одинаковым колличеством строк. Теперь в data_new баланс классов без дублирования

In [6]:
# Функция очистки от лишних символов
def clear_text(corpus):
    white_corpus = []
    for i in range (len(corpus)):
        text = corpus[i]
        text = ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text).split())
        white_corpus.append(text)
    return white_corpus

In [7]:
# lem_text = lemmatize(clear_text(corpus)) 

In [8]:
# разделяем выборку на тренировочную, валидационную и тестовую
features = corpus
target = toxic
features_train, features_valid, target_train, target_valid = train_test_split(corpus, 
                                                                            toxic,
                                                                            test_size=0.1, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid,
                                                                            test_size=0.5, random_state=12345)


In [9]:
# небольшая препдподготовка для функции апсемплинг
features_train = pd.Series(data=features_train)#.reset_index(drop=True)
target_train = pd.Series(target_train)#.reset_index(drop=True)
print(target_train.value_counts())    


0    64687
1     7313
Name: toxic, dtype: int64


In [10]:
# функция для выравнивания классов, так как токсичных коментариев немного. 
def upsample (features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)


    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled


# features_train, target_train = upsample(features_train, target_train, 8)

In [11]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_train, target_train = downsample(features_train, target_train, 0.35)

Функция даунсэмплинга в данном случае более корректна, так как не происходит лишней лемматизации в дальнейшем, что экономит время.

In [12]:
%%time
features_train = lemmatize(clear_text(features_train.reset_index(drop=True)))
print(target_train.value_counts())
# print(features_train)

0    22640
1     7313
Name: toxic, dtype: int64
CPU times: user 2min 33s, sys: 1.06 s, total: 2min 34s
Wall time: 2min 38s


In [13]:
features_valid = lemmatize(clear_text(features_valid.reset_index(drop=True)))
print(target_valid.value_counts())

0    3630
1     370
Name: toxic, dtype: int64


In [14]:
%%time
features_test = lemmatize(clear_text(features_test.reset_index(drop=True)))
print(target_test.value_counts())

0    3561
1     439
Name: toxic, dtype: int64
CPU times: user 21.1 s, sys: 164 ms, total: 21.3 s
Wall time: 21.9 s


In [15]:
# Очищаем наши предложения от стоп-слов

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))



[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что векторизатор был обучен только на тренировочной части данных. Это уменьшает переобучение.
</div>

## Обучение

In [16]:
# Функция для тестирования
def predictions (model):
    pred = model.predict(features_test)
    return f1_score(target_test, pred)

In [17]:
# Pipeline LR
def LR():
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words=stopwords)),
        ('model', LogisticRegression()),
    ])

    f1 = make_scorer(f1_score, greater_is_better=True)
    model = pipeline.fit(features_train, target_train)

#     predicted = model.predict(features_valid)
    f1 = make_scorer(f1_score)
    score = cross_val_score(model, features_train, target_train, scoring=f1, cv=6 )
#     f1 = f1_score(target_valid, predicted)
#     return f1, model
    return score.mean(), model

lr, model_lr = LR()

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [18]:
lr

0.7777307941784265

In [19]:
# Pipeline RFR
def RFR(n_estimators, depth):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words=stopwords)),
        ('model', RandomForestClassifier(n_estimators=n_estimators, max_depth=depth)),
        
    ])
    model = pipeline.fit(features_train, target_train)

    predicted =model.predict(features_test)
    f1 = f1_score(target_test, predicted)

#     f1 = make_scorer(f1_score)
#     score = cross_val_score(model, features_train, target_train, scoring=f1, cv=9 )
    
    return f1, model
#     return score.mean()


rfr, model_rfr = RFR(150,30)

In [20]:
rfr

0.1279317697228145

In [ ]:
def LGBM(max_depth):
    pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords, ngram_range=(2,2))),
                         ('model', LGBMClassifier(max_depth=max_depth, 
                                                                                                     n_estimators=90,
                                                                                                     objective='regression',
                                                                                                     num_leaves=1400,
                                                                                                     metric='f1'))])
    model = pipeline.fit(features_train, target_train)

    predicted = pipeline.predict(features_test)
    f1 = f1_score(target_test, predicted)
#     f1 = make_scorer(f1_score)
#     score = cross_val_score(model, features_train, target_train, scoring=f1, cv=3 )
    return f1, model
#     return score.mean()

lgbm, model_lgbm = LGBM(20)

In [ ]:
f1_all = [['LR', lr], ['RFR', rfr],  ['LGBM', lgbm]]
f1_columns = ['Модель', 'F1']



f1_data = pd.DataFrame(f1_all, columns=f1_columns).pivot_table(index='Модель', values='F1', 
                                                                     aggfunc='sum').sort_values(by='F1')




colors = [ "green", "magenta", "gold"]
f1_data['F1'].plot(kind='bar', color=colors)
print(f1_data)

## Тестирование

In [ ]:
lr_test = predictions(model_lr)
rfr_test = predictions(model_rfr)
lgbm_test = predictions(model_lgbm)

f1_test = [['LR', lr_test], ['RFR', rfr_test],  ['LGBM', lgbm_test]]
f1_columns = ['Модель', 'F1']



f1_data = pd.DataFrame(f1_test, columns=f1_columns).pivot_table(index='Модель', values='F1', 
                                                                     aggfunc='sum').sort_values(by='F1')




colors = [ "green", "magenta", "gold"]
f1_data['F1'].plot(kind='bar', color=colors)
print(f1_data)

## Выводы

В ходе проделанной работы из набора комментариев были удалены возможные дубликаты, сделана выборка для последующего обучения моделей ML. Перед обучением коментарии в выборке были очищены от лишних символов, слова лемматизированны, а также был выровнен баланс классов. 
Из исследуемых моделей ML лучшую заданную метрику f1 на тестовой выборке показала модель логистической регрессии: 0.78, и 0,77 при проведении кросс валидации. У остальных моделей метрика f1 ниже требуемого значения.